# Import

In [1]:
import os

import numpy as np
import pandas as pd
from glob import glob

import nibabel as nib

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# DataLoader

In [4]:
class MyDataset(Dataset):
    def __init__(self, test_size=0.2, test=False):
        
        data_files = glob('../../brainmask_nii/*.nii')
        data_files.sort()
        
        test_num = int(test_size * len(data_files))
        train_num = len(data_files) - test_num
        shuffled_index = np.random.permutation(len(data_files))
        
        train_fname = [data_files[i] for i in shuffled_index[:train_num]]
        test_fname  = [data_files[i] for i in shuffled_index[-test_num:]]
        
        label_file = pd.read_csv('../rsc/age_ixi_and_oasis.csv', index_col=0)
        
        if test:
            self.data_files = [data_files[i] for i in shuffled_index[-test_num:]]
            self.label_file = label_file.age_zscored.values[shuffled_index[-test_num:]]
            
        else:
            self.data_files = [data_files[i] for i in shuffled_index[:train_num]]
            self.label_file = label_file.age_zscored.values[shuffled_index[:train_num]]

    def __getitem__(self, idx):
        #print(idx)
        x = torch.tensor(nib.load(self.data_files[idx]).get_fdata())[None, :, :].float()
        y = torch.tensor(self.label_file[idx]).float()
        return x, y

    def __len__(self):
        return len(self.data_files)

train_dset = MyDataset()
test_dset  = MyDataset(test=True)

train_loader = DataLoader(train_dset, batch_size=8)
test_loader = DataLoader(test_dset, batch_size=8)

# 3D Convolution Model

## Model

In [5]:
class Conv3d(nn.Module):
    
    def __init__(self):
        super(Conv3d, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv3d(1, 8, 5, 3),
            nn.BatchNorm3d(8),
            nn.ReLU(),
            nn.Dropout(.2),
            nn.MaxPool3d(kernel_size=3, stride=3)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv3d(8, 16, 5, 3),
            nn.BatchNorm3d(16),
            nn.ReLU(),
            nn.Dropout(.2),
            nn.MaxPool3d(kernel_size=3, stride=3)
        )
        
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        
        x = x.reshape(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

In [6]:
model = Conv3d().float()
model.to(device)

Conv3d(
  (layer1): Sequential(
    (0): Conv3d(1, 8, kernel_size=(5, 5, 5), stride=(3, 3, 3))
    (1): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): MaxPool3d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv3d(8, 16, kernel_size=(5, 5, 5), stride=(3, 3, 3))
    (1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): MaxPool3d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

## Debug

In [7]:
for i, (x, y)in enumerate(test_loader):
    if i < 1:
        x = x.to(device)
        y = y.to(device)
        print(model.forward(x), y)
    else:
        break

tensor([[-0.5615],
        [ 0.1903],
        [ 0.1420],
        [-0.1417],
        [-0.2647],
        [ 0.6128],
        [-0.0799],
        [-0.0431]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([ 0.2050, -1.7315,  0.9043,  0.2050, -2.0005,  0.3126, -2.0005, -1.8929],
       device='cuda:0')


In [8]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()

In [9]:
epochs = range(1)

trn_losses, tst_losses = [], []

for e in epochs:
    print(f'Starting Epoch {e}')
    
    print(f'Start Training')
    trn_bth_loss = 0
    model.train()
    for i, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()

        y_pred = model.forward(x).to(device)

        loss = loss_fn(y_pred, y)

        loss.backward()
        optimizer.step()

        trn_bth_loss += loss.item()
        
        if not i % 5:
            print(f'{i}th Batch. Loss: {loss.item():.3f}')

    trn_losses.append(trn_bth_loss)
        
        
    tst_bth_loss = 0
    model.eval()
    print(f'Start Testing')
    for i, (x, y) in enumerate(test_loader):
        x, y = x.to(device), y.to(device)

        y_pred = model.forward(x).to(device)
        loss = loss_fn(y_pred, y)
        
        tst_bth_loss += loss.item()

    tst_losses.append(tst_bth_loss)
    
    # Print
    print(f'EPOCHS {e}')
    print(f'TRAIN :: [LOSS] {trn_losses[-1]:.3f}')
    print(f'VALID :: [LOSS] {tst_losses[-1]:.3f}')

Starting Epoch 0
Start Training


C:\Users\pha\anaconda3\envs\bvenv\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0th Batch. Loss: 0.345
5th Batch. Loss: 1.254
10th Batch. Loss: 0.753
15th Batch. Loss: 0.629
20th Batch. Loss: 1.306
25th Batch. Loss: 0.673
30th Batch. Loss: 0.975
35th Batch. Loss: 0.780
40th Batch. Loss: 1.197
45th Batch. Loss: 2.031
50th Batch. Loss: 1.335
55th Batch. Loss: 0.496
60th Batch. Loss: 0.342
65th Batch. Loss: 1.135
70th Batch. Loss: 1.501
75th Batch. Loss: 0.596
80th Batch. Loss: 1.053
85th Batch. Loss: 0.529
90th Batch. Loss: 0.504
95th Batch. Loss: 0.472
100th Batch. Loss: 0.667
105th Batch. Loss: 0.646
110th Batch. Loss: 1.293
115th Batch. Loss: 1.368
Start Testing


RuntimeError: CUDA out of memory. Tried to allocate 146.00 MiB (GPU 0; 3.00 GiB total capacity; 1.58 GiB already allocated; 6.53 MiB free; 1.59 GiB reserved in total by PyTorch)

In [ ]:
y_pred